In [2]:
import os
import sys

# Need to point to where the package is installed
# It will always be in the build directory, but the next folder name might change on your machine
sys.path.append(os.path.abspath('../build/lib.macosx-11.0-arm64-cpython-39'))
import kerrangleconversions as kac

import numpy as np

In [3]:
a = 0.9
p = 10
e = 0.3
x = 0.8
psi = 1.2
chi = 0.5
phi = 10

In [4]:
kac.pyDarwinPhasesToBoyerLindquistPhases(a,p,e,x,psi,chi,phi)

(0.7656854348230792, -0.0866817374910737, 9.264252315901594)